# <span style="color:#007ba7"> 1. O Método Cíclico </span>

## <span style="color:#007ba7"> 1.1. Questão do Negócio </span>

Lembrar do problema inicial que é de um e-commerce de calça jeans masculina.

1. Qual o melhor preço de venda para as calças?
2. Quantos tipos de calças e suas cores para o produto inicial?
3. Quais as matérias primas necessárias para confeccionar as calças?

Tomar os sites da H&M e Macys como referência.

## <span style="color:#007ba7"> 1.2. Ciclo do CRISP-DM </span>

- Estamos na etapa de limpeza de dados. 
- É importante garantir que os dados estejam consistentes e limpos antes de subir para um banco de dados, evitando, assim, enviar sujeiras. 
- Limpar os dados quando já estão no banco de dados é mais complicado do que já subir os dados limpos.

# <span style="color:#007ba7"> 2. Granularidade </span>

É o nível de detalhe na qual o dado é armazenado no banco de dados. 

(Jan L. Harrington, in Relational Database Design and Implementation)

# <span style="color:#007ba7"> 3. Limpeza e definição da granularidade </span>

In [4]:
import re
import pandas as pd 
import numpy as np

In [5]:
# Data
data = pd.read_csv('products_hm.csv')
data = data.drop(['Unnamed: 0'], axis=1)
data = data[['product_id_x', 'product_category', 'product_name', 'product_price', 'scrapy_datetime', 'style_id', 'color_id', 'color_name',
              'Size', 'Fit', 'Composition', 'More sustainable materials']].copy()
# Changing column names
data.columns = ['product_id', 'product_category', 'product_name', 'product_price',
       'scrapy_datetime', 'style_id', 'color_id', 'color_name', 'size', 'fit',
       'composition', 'product_safety']

# product_name
data['product_name'] = data['product_name'].apply(lambda x: x.replace(' ', '_').lower())

# product_price
data['product_price'] = data['product_price'].apply(lambda x: x.replace('$ ', '')).astype(float)

# scrapy_datetime
data['scrapy_datetime'] = pd.to_datetime(data['scrapy_datetime'], format='%Y-%m-%d %H:%M:%S')

# style_id
data['style_id'] = data['style_id'].astype(int)


# color_id
data['color_id'] = data['color_id'].astype(int)

# color_name
data['color_name'] = data['color_name'].apply(lambda x: x.replace(' ', '_').lower())

# Fit
data['fit'] = data['fit'].apply(lambda x: x.replace(' ', '_').lower())

# size number
data['size_number'] = data['size'].apply(lambda x: re.search ('\d{3}cm', x).group(0) if pd.notnull(x) else x)
data['size_number'] = data['size'].apply(lambda x: re.search('\d+', x).group(0) if pd.notnull(x) else x)

# size model
data['size_model'] = data['size'].str.extract('(\d+/\\d+)')

# removing the size variable, because we have already removed the information we wanted
data = data.drop(columns=['size', 'product_safety'], axis = 1)

# product_composition
# Excluding composition I don't want for the first cycle of CRISP
data = data[~data['composition'].str.contains('Pocket lining:', na=False)]
data = data[~data['composition'].str.contains('Lining:', na=False)]
data = data[~data['composition'].str.contains('Shell:', na=False)]
data = data[~data['composition'].str.contains('Pocket:', na=False)]

# reset_index
data = data.reset_index()

# composition 
# cotton | polyester | spandex |
df_ref = pd.DataFrame(index=np.arange(len(data)), columns = ['cotton', 'spandex', 'polyester'])

# break composition by comma
df1 = data['composition'].str.split(',', expand = True)

# cotton
df_cotton = df1[0]
df_cotton.name = 'cotton'
df_ref = pd.concat([df_ref, df_cotton], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
df_ref['cotton'] = df_ref['cotton'].fillna('Cotton 0%')

# polyester
df_polyester = df1.loc[df1[1].str.contains('Polyester', na=True), 1]
df_polyester.name = 'polyester'

df_ref = pd.concat([df_ref, df_polyester], axis = 1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
df_ref['polyester'] = df_ref['polyester'].fillna('Polyester 0%')

# spandex
df_spandex = df1.loc[df1[1].str.contains('Spandex', na=True), 1]
df_spandex.name = 'spandex'

# combine spandex from both columns 1 and 2
df_spandex = df_spandex.combine_first(df1[2])

df_ref = pd.concat([df_ref, df_spandex], axis = 1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]
df_ref['spandex'] = df_ref['spandex'].fillna('Spandex 0%')

# final join
data = pd.concat([data, df_ref], axis=1)

# Format composition data
data['cotton'] = data['cotton'].apply(lambda x: re.search('\d+', x).group(0) if pd.notnull(x) else x)
data['polyester'] = data['polyester'].apply(lambda x: re.search('\d+', x).group(0) if pd.notnull(x) else x)
data['spandex'] = data['spandex'].apply(lambda x: re.search('\d+', x).group(0) if pd.notnull(x) else x)

# Converting composition components to int
data['cotton'] = data['cotton'].astype(int)
data['polyester'] = data['polyester'].astype(int)
data['spandex'] = data['spandex'].astype(int)

# Drop  columns
data = data.drop(columns=['index', 'composition'])

# Drop duplicates
data = data.drop_duplicates()

In [6]:
# Salvando a base de dados limpa em csv
data.to_csv('products_hm_cleaned.csv', index=False)